In [9]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../CommonUtils/') # https://github.com/glukicov/EDMTracking/tree/master/CommonUtils
import CommonUtils as cu
font_size=16

In [10]:
data = pd.read_csv("../DATA/scans/scan.csv")
par_n=-1
if(data.shape[1] == 27):  par_n=10
if(data.shape[1] == 25):  par_n=9
if(data.shape[1] == 17):  par_n=5
print("par_n =", par_n, "according to expected total columns")

par_n = 9 according to expected total columns


In [11]:
# iterators
stations=(12, 18)
dss = ("60h", "9D", "HK", "EG")

In [12]:
#iterative cuts
station=stations[1]
ds=dss[0]

In [13]:
# apply cuts and select data
station_cut = (data["station"]==station)
ds_cut =  (data['ds']==ds)
plot_data= data[station_cut & ds_cut]
plot_data=plot_data.reset_index()

In [14]:
# resolve paramters for plotting 
start=plot_data['start']
stop=plot_data['stop']
chi2 = plot_data['chi2']
chi2_e=np.sqrt(2/ (plot_data['ndf']-par_n) )

In [15]:
# get for plotting
x=start
y=chi2
y_e=chi2_e 
y_s = np.sqrt(y_e**2-y_e[0]**2) # 1sigma band

In [16]:
fig, ax = cu.plot(x, y, y_err=y_e, error=True, elw=2, label="S"+str(station)+": "+ds+" DS")
ax.plot(x, y, marker=".", ms=10, c="g", lw=0)
ax.plot(x, y[0]+y_s, c="r", ls=":", lw=2, label=r"$\sigma_{\Delta_{21}}$")
ax.plot(x, y[0]-y_s, c="r", ls=":", lw=2)
ax.plot([min(x)-2, max(x)+2], [1, 1], c="k", ls="--")
ax.set_xlim(min(x)-2, max(x)+2)
ax.set_xlabel(r"Start time [$\rm{\mu}$s]", fontsize=font_size);
ax.set_ylabel(r"$\frac{\chi^2}{\rm{DoF}}$", fontsize=font_size);
ax.legend(fontsize=font_size, loc='upper center', bbox_to_anchor=(0.5, 1.1))
fig.savefig("../fig/start_chi2_S"+str(station)+".png", dpi=300)

In [17]:
par_name = 'tau'

In [18]:
y=plot_data[par_name]
y_e=plot_data[par_name+'_e']
y_s = np.sqrt(y_e**2-y_e[0]**2) # 1sigma band

In [19]:
fig, ax = cu.plot(x, y, y_err=y_e, error=True, elw=2, label="S"+str(station)+": "+ds+" DS")
ax.plot(x, y, marker=".", ms=10, c="g", lw=0)
ax.plot(x, y[0]+y_s, c="r", ls=":", lw=2, label=r"$\sigma_{\Delta_{21}}$")
ax.plot(x, y[0]-y_s, c="r", ls=":", lw=2)
if(par_name=='tau'): ax.plot([np.min(x)-2, np.max(x)+2], [64.44, 64.44], c="k", ls="--"); ax.set_ylim(np.min(y)-0.1, 64.6)
ax.set_xlim(min(x)-2, max(x)+2)
ax.set_xlabel(r"Start time [$\rm{\mu}$s]", fontsize=font_size);
ax.set_ylabel(r"$\tau$ [$\rm{\mu}$s]", fontsize=font_size);
ax.legend(fontsize=font_size, loc='upper center', bbox_to_anchor=(0.5, 1.1))
fig.savefig("../fig/start_"+par_name+"_"+str(station)+".png", dpi=300)

In [20]:
import sys, re, subprocess
import argparse
import pandas as pd
import numpy as np
np.set_printoptions(precision=3) # 3 sig.fig 
sys.path.append('../CommonUtils/') # https://github.com/glukicov/EDMTracking/blob/master/CommonUtils/CommonUtils.py
import CommonUtils as cu
from scipy import stats, optimize, fftpack
import matplotlib as mpl
mpl.use('Agg') # MPL in batch mode
font_size=16
import matplotlib.pyplot as plt
import seaborn as sn


In [21]:
stations=(12, 18)
# dss = ("60h", "9D", "HK", "EG")
dss = ("60h")
# par_names=["N", "tau", "A", "R", "phi", "A_cbo", "w_cbo", "phi_cbo", "tau_cbo", "K_LM"]
par_names=["N", "tau", "A", "R", "phi", "A_cbo", "w_cbo", "phi_cbo", "tau_cbo"]
# par_label=[r"$N$", r"$\tau$", r"$A$", r"$R$", r"$\phi$", r"$A_{\rm{CBO}}$", r"$\omega_{\rm{CBO}}$", r"$\phi_{\rm{CBO}}$", r"$\tau_{\rm{CBO}}$", r"$\K_{\rm{LM}}$"])
par_label=[r"$N$", r"$\tau$", r"$A$", r"$R$", r"$\phi$", r"$A_{\rm{CBO}}$", r"$\omega_{\rm{CBO}}$", r"$\phi_{\rm{CBO}}$", r"$\tau_{\rm{CBO}}$"]
direction="start"

In [22]:
data = pd.read_csv("../DATA/scans/scan.csv")
par_n=-1
if(data.shape[1] == 27):  par_n=10
if(data.shape[1] == 25):  par_n=9
if(data.shape[1] == 17):  par_n=5
print("par_n =", par_n, "according to expected total columns")
if(par_n!=len(par_names)): raise Exception("More parameters in scan data then expected names - expand!")
if(par_n==--1): raise Exception("Scan data has less/more then expected parameters!")

par_n = 9 according to expected total columns


In [23]:
data['A_cbo']

0    -0.026226
1     0.034313
2     0.029268
3     0.017496
4    -0.030784
        ...   
66   -0.011664
67   -0.023907
68   -0.011407
69   -0.018025
70   -0.011674
Name: A_cbo, Length: 71, dtype: float64

In [24]:
plot_data

,index,start,A,A_cbo,A_cbo_e,A_e,N,N_e,R,R_e,...,phi_cbo_e,phi_e,station,stop,tau,tau_cbo,tau_cbo_e,tau_e,w_cbo,w_cbo_e
0,1,30.287600,0.340372,0.034313,0.003468,0.000783,12762.621444,13.541873,-96.167029,26.870364,...,0.101303,0.004328,18,450.0,63.698832,1.167170e+02,2.012994e+01,0.038426,2.326986,0.001481
1,3,32.279383,0.341077,0.017496,0.000910,0.000809,12750.099921,14.012063,-93.728406,27.568045,...,0.088684,0.004506,18,450.0,63.722280,-1.418543e+09,1.041209e+14,0.039378,2.330526,0.000783
2,5,34.271166,0.340843,0.033772,0.003736,0.000800,12736.933260,14.206640,-75.749607,27.321978,...,0.111513,0.004522,18,450.0,63.755600,1.208988e+02,2.225843e+01,0.039286,2.328071,0.001532
3,7,36.262949,0.341700,0.031158,0.003568,0.000813,12718.677913,14.454350,-74.630135,27.548431,...,0.113688,0.004630,18,450.0,63.789218,1.370708e+02,2.797581e+01,0.039603,2.329316,0.001483
4,9,38.254731,0.341423,0.030879,0.003779,0.000831,12704.663256,15.114020,-75.397156,28.281120,...,0.123831,0.004819,18,450.0,63.813305,1.381057e+02,2.959252e+01,0.040799,2.329494,0.001563
5,11,40.246514,0.341556,0.032679,0.004189,0.000836,12702.756564,15.337082,-84.237893,28.420736,...,0.127061,0.004916,18,450.0,63.823819,1.282383e+02,2.657296e+01,0.040908,2.329498,0.001609
6,13,42.238297,0.341915,0.027709,0.003598,0.000842,12690.745116,15.725993,-48.040357,28.718014,...,0.131620,0.005045,18,450.0,63.844117,1.660640e+02,4.154288e+01,0.041386,2.330814,0.001519
7,15,44.230080,0.342128,0.025361,0.003541,0.000863,12676.935420,16.384777,-72.336702,29.568168,...,0.138653,0.005274,18,450.0,63.867031,1.905248e+02,5.554538e+01,0.042598,2.330967,0.001525
8,18,48.213646,0.342572,0.029338,0.004468,0.000874,12660.692041,17.123674,-61.518656,30.036125,...,0.151101,0.005508,18,450.0,63.901745,1.530077e+02,3.937040e+01,0.043386,2.330749,0.001675
9,20,50.205429,0.343387,0.026704,0.004212,0.000900,12639.929952,17.676986,-57.819145,30.758041,...,0.159225,0.005724,18,450.0,63.937095,1.749160e+02,5.102181e+01,0.044403,2.331368,0.001680


0

In [20]:
print("S",station,":", end =" ")
station_cut = (data["station"]==station)
ds_cut =  (data['ds']==ds)
plot_data=data[station_cut & ds_cut]
plot_data=plot_data.reset_index()

IndentationError: unexpected indent (<ipython-input-20-c6f6bf06e020>, line 2)